In [ ]:
import os
import numpy as np
import nibabel as nib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout

# Define paths
train_dir = r"D:\Parkinson\train set"

# List all training image files
train_files = [os.path.join(train_dir, file) for file in os.listdir(train_dir)]

# Load and preprocess training images
def load_images(file_paths):
    images = []
    for file_path in file_paths:
        # Load NIfTI image using nibabel
        image = nib.load(file_path).get_fdata()
        # Normalize or preprocess image (you can add more preprocessing steps)
        image = (image - np.min(image)) / (np.max(image) - np.min(image))
        images.append(image)
    return np.array(images)

# Load training images
train_images = load_images(train_files)

# Reshape train_images to include the channels dimension (assuming single-channel input)
train_images = train_images[..., np.newaxis]

# Define the number of classes based on your labels
num_classes = 2  # Adjust this based on your specific classification task
train_labels = np.random.randint(0, num_classes, size=len(train_files))

# Define a simpler 3D CNN model with reduced complexity
model = Sequential([
    Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=train_images.shape[1:]),
    MaxPooling3D(pool_size=(2, 2, 2)),
    Conv3D(64, kernel_size=(3, 3, 3), activation='relu'),
    MaxPooling3D(pool_size=(2, 2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

# Train the model
num_epochs = 10
batch_size = 4
model.fit(train_images, train_labels, epochs=num_epochs, batch_size=batch_size)
model.save("3d_cnn_model1.h5")
loss, accuracy = model.evaluate(train_images, train_labels)
print(f"Training Accuracy: {accuracy * 100:.2f}%")


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 179, 215, 179, 32  896       
                             )                                   
                                                                 
 max_pooling3d (MaxPooling3D  (None, 89, 107, 89, 32)  0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 87, 105, 87, 64)   55360     
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 43, 52, 43, 64)   0         
 3D)                                                             
                                                                 
 flatten (Flatten)           (None, 6153472)           0         
                                                        

In [ ]:
import os
import numpy as np
import nibabel as nib
from tensorflow.keras.models import load_model

# Define paths
test_dir = r"D:\Parkinson\test set"

# List all test image files
test_files = [os.path.join(test_dir, file) for file in os.listdir(test_dir)]

# Load and preprocess test images
def load_images(file_paths):
    images = []
    for file_path in file_paths:
        # Load NIfTI image using nibabel
        image = nib.load(file_path).get_fdata()
        # Normalize or preprocess image (you can add more preprocessing steps)
        image = (image - np.min(image)) / (np.max(image) - np.min(image))
        images.append(image)
    return np.array(images)

# Load test images
test_images = load_images(test_files)

# Reshape test_images to include the channels dimension (assuming single-channel input)
test_images = test_images[..., np.newaxis]

# Load the saved model
saved_model = load_model("3d_cnn_model1.h5")

# Predict labels for test images
predictions = saved_model.predict(test_images)


num_classes = 2
test_labels = np.random.randint(0, num_classes, size=len(test_files))

# Calculate accuracy
correct_predictions = np.argmax(predictions, axis=1) == test_labels
accuracy = np.mean(correct_predictions)

print(f"Test Accuracy: {accuracy * 100:.2f}%")
